In [1]:
import requests
from bs4 import BeautifulSoup

def get_grid_conditions(grid_num: int | None = None) -> list[str]:
    base_url = 'https://www.immaculategrid.com'
    var_url = f"/grid-{grid_num}"
    url = base_url + var_url if grid_num else base_url
    response = requests.get(url)
    if response.status_code != 200:
        return []
    soup = BeautifulSoup(response.content, 'html.parser')
    condition_pairs = [button.get('aria-label') for button in soup.find_all('button', class_='focus:bg-yellow-200 w-full h-full')]
    ret = []
    for pair in condition_pairs:
        cond1, cond2 = pair.split(" + ")
        cond1 = " ".join(cond1.split())
        cond2 = " ".join(cond2.split())
        ret.append([cond1, cond2])
    return ret

def get_max_grid_number() -> int:
    url = 'https://www.immaculategrid.com'

    response = requests.get(url)
    if response.status_code != 200:
        return -1
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    grid_num = int(
        soup
        .find('div', class_="uppercase font-medium")
        .text
        .split(" ")[-1][1:])
    
    return grid_num

def backfill_IM_conditions(limit: int = 10) -> list[list[str]]:
    max_grid_num = get_max_grid_number()
    return [
        get_grid_conditions(grid)
        for grid in reversed(range(max_grid_num - limit + 1, max_grid_num + 1))
    ]
        

In [5]:
grid_backfill = backfill_IM_conditions(limit=get_max_grid_number())

In [6]:
st = set()
for date in grid_backfill:
    for row in date:
        print(row)
        for cond in row:
            st.add(cond)
lst_st = list(st)

['Los Angeles Angels', 'Pittsburgh Pirates']
['Los Angeles Angels', 'Baltimore Orioles']
['Los Angeles Angels', 'Played Shortstop min. 1 game']
['New York Yankees', 'Pittsburgh Pirates']
['New York Yankees', 'Baltimore Orioles']
['New York Yankees', 'Played Shortstop min. 1 game']
['.300+ AVG Season Batting', 'Pittsburgh Pirates']
['.300+ AVG Season Batting', 'Baltimore Orioles']
['.300+ AVG Season Batting', 'Played Shortstop min. 1 game']
['Oakland Athletics', 'Boston Red Sox']
['Oakland Athletics', 'Kansas City Royals']
['Oakland Athletics', 'Born Outside US 50 States and DC']
['Houston Astros', 'Boston Red Sox']
['Houston Astros', 'Kansas City Royals']
['Houston Astros', 'Born Outside US 50 States and DC']
['Washington Nationals', 'Boston Red Sox']
['Washington Nationals', 'Kansas City Royals']
['Washington Nationals', 'Born Outside US 50 States and DC']
['Atlanta Braves', 'San Diego Padres']
['Atlanta Braves', 'Chicago White Sox']
['Atlanta Braves', '2000+ Hits Career Batting']
['M

In [74]:
sorted(lst_st)

['.300+ AVG Career Batting',
 '.300+ AVG Season Batting',
 '10+ HR Season Batting',
 '10+ Win Season Pitching',
 '100+ RBI Season Batting',
 '100+ Run Season Batting',
 '20+ Win Season Pitching',
 '200+ Hits Season Batting',
 '200+ K Season Pitching',
 '200+ Wins Career Pitching',
 '2000+ Hits Career Batting',
 '2000+ K Career Pitching',
 '30+ HR / 30+ SB Season Batting',
 '30+ HR Season Batting',
 '30+ SB Season',
 '30+ Save Season Pitching',
 '300+ HR Career Batting',
 '300+ Save Career Pitching',
 '300+ Wins Career Pitching',
 '3000+ Hits Career Batting',
 '3000+ K Career Pitching',
 '40+ 2B Season Batting',
 '40+ HR Season Batting',
 '40+ Save Season Pitching',
 '40+ WAR Career',
 '500+ HR Career Batting',
 '6+ WAR Season',
 'All Star',
 'Arizona Diamondbacks',
 'Atlanta Braves',
 'Baltimore Orioles',
 'Born Outside US 50 States and DC',
 'Boston Red Sox',
 'Canada',
 'Chicago Cubs',
 'Chicago White Sox',
 'Cincinnati Reds',
 'Cleveland Guardians',
 'Colorado Rockies',
 'Cy Young',

In [14]:
from fastapi import HTTPException, status
import os
import pymysql.cursors
from pymysql import converters


class DatabaseConnector:
    def __init__(self, db: str = "immaculateGrid"):
        self.host = os.getenv("DATABASE_HOST")
        self.user = os.getenv("DATABASE_USERNAME")
        self.password = os.getenv("DATABASE_PASSWORD")
        self.database = db
        self.port = int(os.getenv("DATABASE_PORT"))
        self.conversions = converters.conversions
        self.conversions[pymysql.FIELD_TYPE.BIT] = (
            lambda x: False if x == b"\x00" else True
        )
        if not self.host:
            raise EnvironmentError("DATABASE_HOST environment variable not found")
        if not self.user:
            raise EnvironmentError("DATABASE_USERNAME environment variable not found")
        if not self.password:
            raise EnvironmentError("DATABASE_PASSWORD environment variable not found")
        if not self.database:
            raise EnvironmentError("DATABASE environment variable not found")

    def get_connection(self):
        connection = pymysql.connect(
            host=self.host,
            port=self.port,
            user=self.user,
            password=self.password,
            database=self.database,
            cursorclass=pymysql.cursors.DictCursor,
            conv=self.conversions,
        )
        return connection

    def query_get(self, sql, param=None):
        try:
            connection = self.get_connection()
            with connection:
                with connection.cursor() as cursor:
                    cursor.execute(sql, param)
                    return cursor.fetchall()
        except Exception as e:
            raise HTTPException(
                status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
                detail="Database error: " + str(e),
            )

    def query_put(self, sql, param):
        try:
            connection = self.get_connection()
            with connection:
                with connection.cursor() as cursor:
                    cursor.execute(sql, param)
                    connection.commit()
                    return cursor.lastrowid
        except Exception as e:
            raise HTTPException(
                status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
                detail="Database error: " + str(e),
            )


In [48]:
from typing import Union
from pydantic import BaseModel

class BaseResponse(BaseModel):
    pass

class Team(BaseResponse):
    team_id: str

class Position(BaseResponse):
    position_id: str

class Statistic(BaseResponse):
    timeframe: str
    type: str
    stat: str
    value: int

class Award(BaseResponse):
    award: str

class PlayerAttribute(BaseResponse):
    attribute: str

class Other(BaseResponse):
    other: str


In [39]:
import os

os.environ['DATABASE_PORT'] = "3306"
os.environ['DATABASE_HOST'] = "localhost"
os.environ['DATABASE_USERNAME'] = "root"
os.environ['DATABASE_PASSWORD'] = "P4ssw0rd"

db = DatabaseConnector()


In [55]:
def parse_teamID(possible_team_name: str) -> Team | None:
    if possible_team_name == "Los Angeles Angels":
        return Team(team_id="LAA")

    lk_query = """
        SELECT teamID, teamName 
        FROM immaculateGrid.TeamAttribute
        WHERE year = 2022;
        """
    team_lk_dict = {
        team["teamID"]: team["teamName"]
        for team in db.query_get(lk_query)
    }
    
    id = team_lk_dict.get(possible_team_name, "")
    return Team(team_id=id) if id else None


def parse_postions(possible_pos_cond: str) -> Position | None:
    position_dict = {
        'Pitched min. 1 game': "P",
        'Played Catcher min. 1 game': "C", 
        'Played First Base min. 1 game': "1B", 
        'Played Second Base min. 1 game': "2B", 
        'Played Third Base min. 1 game': "3B",
        'Played Shortstop min. 1 game': "SS", 
        'Played Left Field min. 1 game': "LF", 
        'Played Center Field min. 1 game': "CF", 
        'Played Right Field min. 1 game': "RF",       
        'Played Outfield min. 1 game': "OF", 
        'Designated Hitter min. 1 game': "DH",
    }
    pos_key = position_dict.get(possible_pos_cond, "")
    if pos_key:
        return Position(position_id=pos_key)


In [57]:
parse_teamID("Played Catcher min. 1 game")

In [58]:
parse_postions("Played Catcher min. 1 game")

Position(position_id='C')

In [64]:
import re

def parse_statistic(description: str) -> Statistic:
    # Patterns to capture parts of the statistic
    pattern = re.compile(r'(\d+|\≤\d+)\s*(HR|SB|ERA|Hits|K|Wins|WAR|RBI|Run)\s*(Career|Season)\s*(Batting|Pitching)')
    match = pattern.match(description)
    
    if not match:
        return None
    
    value_str, stat_name, timeframe, stat_type = match.groups()
    
    # Clean value
    value = int(value_str.replace('≤', '').replace('+', ''))
    
    return Statistic(
        timeframe=timeframe,
        type=stat_type,
        stat=stat_name,
        value=value
    )

In [67]:
parse_statistic("≤ 3.00 ERA Career Pitching")

In [59]:
team_lk = db.query_get("""
        SELECT teamID, teamName 
        FROM immaculateGrid.TeamAttribute
        WHERE year = 2022;
        """)


{'ARI': 'Arizona Diamondbacks',
 'ATL': 'Atlanta Braves',
 'BAL': 'Baltimore Orioles',
 'BOS': 'Boston Red Sox',
 'CHA': 'Chicago White Sox',
 'CHN': 'Chicago Cubs',
 'CIN': 'Cincinnati Reds',
 'CLE': 'Cleveland Guardians',
 'COL': 'Colorado Rockies',
 'DET': 'Detroit Tigers',
 'HOU': 'Houston Astros',
 'KCA': 'Kansas City Royals',
 'LAA': 'Los Angeles Angels of Anaheim',
 'LAN': 'Los Angeles Dodgers',
 'MIA': 'Miami Marlins',
 'MIL': 'Milwaukee Brewers',
 'MIN': 'Minnesota Twins',
 'NYA': 'New York Yankees',
 'NYN': 'New York Mets',
 'OAK': 'Oakland Athletics',
 'PHI': 'Philadelphia Phillies',
 'PIT': 'Pittsburgh Pirates',
 'SDN': 'San Diego Padres',
 'SEA': 'Seattle Mariners',
 'SFN': 'San Francisco Giants',
 'SLN': 'St. Louis Cardinals',
 'TBA': 'Tampa Bay Rays',
 'TEX': 'Texas Rangers',
 'TOR': 'Toronto Blue Jays',
 'WAS': 'Washington Nationals'}